In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

In [2]:
class Identity(nn.Module):
  def __init__(self):
    super(Identity, self).__init__()

  def forward(self, x):
    return x

In [3]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
in_channels=3
num_classes=10
learning_rate=1e-4
batch_size=1024
num_epochs=5

In [5]:
train_dataset=datasets.CIFAR10(root='dataaet/', train=True, transform=transforms.ToTensor(), download=True)
train_loader= DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#test_dataset= datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
#test_loader= DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting dataaet/cifar-10-python.tar.gz to dataaet/


In [22]:
model=torchvision.models.vgg16(pretrained=True)

for param in model.parameters():
  param.requires_grad=False

model.avgpool=Identity()
model.classifier=nn.Linear(512,10)
#model.classifier=nn.Sequential(nn.Linear(512,10),nn.ReLU(),nn.Linear(100,10))
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [17]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
for epoch in range(num_epochs):
  losses=[]

  for batch_idx, (data, targets) in enumerate(train_loader):
    data=data.to(device=device)
    targets=targets.to(device=device)

    scores=model(data)
    loss=criterion(scores, targets)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

  mean_loss=sum(losses)/len(losses)
  print(f'loss at epoch (epoch) was {mean_loss:.5f}')

loss at epoch (epoch) was 2.45384
loss at epoch (epoch) was 2.45389
loss at epoch (epoch) was 2.45395
loss at epoch (epoch) was 2.45385
loss at epoch (epoch) was 2.45389


In [24]:
def check_accuracy(loader, model):
  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device=device)
      y=y.to(device=device)

      scores=model(x)
      _, predictions= scores.max(1)
      num_correct+= (predictions==y).sum()
      num_samples+= predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()

In [25]:
check_accuracy(train_loader, model)
#check_accuracy(test_loader, model)

Got 4419 / 50000 with accuracy 8.84
